In [ ]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FilePurpose, CodeInterpreterTool, ListSortOrder, MessageRole

In [ ]:
load_dotenv()

In [ ]:
# Create agent client
credential = DefaultAzureCredential(exclude_environment_credential=True,
         exclude_managed_identity_credential=True)
project_client = AIProjectClient(
    credential=credential,
    endpoint=os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT"),
)
project_client 

### Configure Azure AI Search index as a tool
Using the connection ID of your Azure AI Search resource, configure the Azure AI Search tool to use your Azure AI Search index.

In [ ]:
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType
from azure.ai.projects.models import ConnectionType

# Define Azure AI Search tool connection id and index name

ai_search_conn_id = project_client.connections.get_default(ConnectionType.AZURE_AI_SEARCH).id

index_name = "rgr-rel-notes-index"

ai_search_tool = AzureAISearchTool(
    index_connection_id=ai_search_conn_id,
    index_name=index_name,
    query_type=AzureAISearchQueryType.VECTOR,
    filter=None,
    top_k=3
)

ai_search_tool.resources

In [ ]:
# dir(AzureAISearchQueryType)
ai_search_tool.definitions

In [ ]:
# Define an agent that uses the code ai_search tool
agent = project_client.agents.create_agent(
    name="TicketAnalyzer-Agent",
    model=os.environ["GPT_MODEL_DEPLOYMENT_NAME"],  # Model deployment name 4o
    instructions="Answer the user's questions about Digital Health Solution based on the added knowledge. Include page # from the knowledge docs as a reference.",
    tools=ai_search_tool.definitions,
    tool_resources=ai_search_tool.resources,
)

print(f"Created agent, ID: {agent.id}")

In [ ]:
q1 = "Describe the document"
q2 = "Extract the known issues details"
q3 = "Extract the known issues details with issue id and description in JSON format"

In [ ]:
from azure.ai.agents.models import MessageRole, ListSortOrder

# create a thread for the communication with the agent
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# add a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content=q1,
)

print(f"Added message, ID: {message.id}")

### Create and process a run with the specified thread and agent

In [ ]:
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)
print(f"Run finished with status: {run.status}")

# Check if the run failed
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

### Fetch and log all messages in the thread

In [ ]:
messages = project_client.agents.messages.list(
    thread_id=thread.id,
    order=ListSortOrder.ASCENDING
)

for msg in messages:
    print(f"{msg.role}> {msg.content}")

### Continuous Evaluations against predefined evaluators 
https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/observability#what-are-evaluators

In [ ]:
from azure.ai.projects.models import EvaluatorIds

evaluators={
    "Relevance": {"Id": EvaluatorIds.Relevance.value},
    "Fluency": {"Id": EvaluatorIds.Fluency.value},
    "Coherence": {"Id": EvaluatorIds.Coherence.value},
    # "Groundedness": {"Id": EvaluatorIds.Groundedness.value},
}
dir(EvaluatorIds)

### Continuously evaluate your agent run by creating an AgentEvaluationRequest

In [ ]:
from azure.ai.projects.models import AgentEvaluationRequest

dir(AgentEvaluationRequest)

# Create an AgentEvaluationRequest to evaluate the agent run
eval_request = AgentEvaluationRequest(  
        thread_id=thread.id,
        run_id=run.id,   
        evaluators=evaluators,
        app_insights_connection_string=project_client.telemetry.get_connection_string(),
    )



In [ ]:
# project_client.evaluations.create_agent_evaluation(
#     eval_request
# )

In [ ]:
dir(project_client.evaluations)

In [ ]:
evaluation = project_client.evaluations.get(name='thread_Xcv3cjn5kAMTJYOix0Z4v7w9;run_bwu832yz5HrPsNtGcUP2uuA5')
evaluation